# Demo for the SMILES processing

In [ ]:
import sys
sys.path.append('/mnt/d/Research/PHD/DLEPS/code/DLEPS')
import molecule_vae
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem import Draw

import numpy as np  
import pandas as pd

In [6]:
dt1 = pd.read_csv('/mnt/d/Research/PHD/DLEPS/results/train_SMILES_demo.csv')
dt2 = pd.read_csv('/mnt/d/Research/PHD/DLEPS/results/test_SMILES_demo.csv')

smiles = np.concatenate([dt1['smiles'].values, dt2['smiles'].values],axis=0) 
    

In [ ]:
smiles

In [ ]:
#list(smiles)

In [ ]:
print(smiles[2])
MolToSmiles(MolFromSmiles(smiles[2]))

In [7]:
#smiles = smiles[:100]
smiles_rdkit = []
iid = []
for i in range(len(smiles)):
    try:
        smiles_rdkit.append(MolToSmiles(MolFromSmiles(smiles[ i ])))
        iid.append(i)
        #print(i)
    except:
        print("Error at %d" % (i))
        

In [8]:
print(len(smiles))
print(len(iid))

8889
8889


In [ ]:
#smiles_rdkit

In [ ]:
def get_zinc_tokenizer(cfg):
    long_tokens = [a for a in list(cfg._lexical_index.keys()) if xlength(a) > 1] ####
    replacements = ['$','%','^'] # ,'&']
    assert xlength(long_tokens) == len(replacements) ####xzw
    for token in replacements: 
        assert token not in cfg._lexical_index ####
    
    def tokenize(smiles):
        for i, token in enumerate(long_tokens):
            smiles = smiles.replace(token, replacements[i])
        tokens = []
        for token in smiles:
            try:
                ix = replacements.index(token)
                tokens.append(long_tokens[ix])
            except:
                tokens.append(token)
        return tokens
    
    return tokenize


In [ ]:
from functools import reduce

def xlength(y):
    return reduce(lambda sum, element: sum + 1, y, 0)


In [ ]:
import zinc_grammar
import nltk

_tokenize = get_zinc_tokenizer(zinc_grammar.GCFG)
_parser = nltk.ChartParser(zinc_grammar.GCFG)
_productions = zinc_grammar.GCFG.productions()
_prod_map = {}
for ix, prod in enumerate(_productions):
    _prod_map[prod] = ix
MAX_LEN = 277
_n_chars = len(_productions)


In [ ]:
smiles_rd = smiles_rdkit[:100]

In [ ]:
        """ Encode a list of smiles strings into the latent space """
        assert type(smiles_rdkit) == list
        tokens = map(_tokenize, smiles_rdkit)
        parse_trees = []
        i = 0
        badi = []
        for t in tokens:
            #while True:
            try:
                tp = next(_parser.parse(t))
                parse_trees.append(tp)
            except:
                print("Parse tree error at %d" % i)
                badi.append(i)
            i += 1
            #print(i)
        productions_seq = [tree.productions() for tree in parse_trees]
        indices = [np.array([_prod_map[prod] for prod in entry], dtype=int) for entry in productions_seq]
        one_hot = np.zeros((len(indices), MAX_LEN, _n_chars), dtype=np.float32)
        for i in range(len(indices)):
            num_productions = len(indices[i])
            if num_productions > MAX_LEN:
                print("Too large molecules, out of range")
            #print("i=  {%d} len(indices)=  {%d} num_productions = %d " % (i,len(indices),num_productions))
                one_hot[i][np.arange(MAX_LEN),indices[i][:MAX_LEN]] = 1.
            else:    
                one_hot[i][np.arange(num_productions),indices[i]] = 1.
                one_hot[i][np.arange(num_productions, MAX_LEN),-1] = 1.

In [ ]:
iix = np.arange(len(smiles_rdkit))
iidx = [i for i in iix if i not in badi]
len(iidx)
iid2 = np.array(iid)[iidx]
len(iid2)

In [ ]:
L962 = np.genfromtxt('/mnt/d/Research/PHD/DLEPS/results/L1000_landmark.csv', delimiter=',')
L962 = L962[iid2]
print(L962.shape)
num_examples = L962.shape[0]

In [ ]:
perm = np.arange(num_examples)
np.random.shuffle(perm)
#L962 = L962[perm]
smile_rd = one_hot[perm]
TEST_SIZE = 3000
#L962_train = L962[TEST_SIZE:]
smile_train = smile_rd[TEST_SIZE:]
#L962_test = L962[:TEST_SIZE]
smile_test = smile_rd[:TEST_SIZE]

In [ ]:
savetxt(fname, X, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)[source]
np.savetxt("/mnt/d/Research/PHD/DLEPS/results/L1000_train.csv",L962_train, delimiter = ',')
np.savetxt("/mnt/d/Research/PHD/DLEPS/results/L1000_test.csv",L962_test, delimiter = ',')

In [ ]:
import h5py

h5f = h5py.File('SMILE_train_demo.h5', 'w')
h5f.create_dataset('data', data=smile_train)
h5f.close()

h5f = h5py.File('SMILE_test_demo.h5', 'w')
h5f.create_dataset('data', data=smile_test)
h5f.close()

